In [ ]:
#in questo quaderno verranno valutati i modelli migliori su alcuni dataset di benchmark

In [ ]:
#il primo dataset scelto è SemEval_2017 task 4, in quanto è composto da tweet, ossia frasi brevi etichettate come pos/neu/neg
#gli altri file hanno reviews troppo lunghe e etichettate pos(neg e basta)

In [ ]:
#importa sem eveal test
#sposta cartelle modelli vincitori qui
#crea funzione univoca per valutazione
#scrivi email con risultati ai prof

In [2]:
import pandas as pd
sem_eval_test=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_test_polarity\\Evaluation_data\\Sem_Eval_EN\\sem_eval_test.csv")
sem_eval_test


,Unnamed: 0,Sentence,Polarity_Classification
0,0,#ArianaGrande Ari By Ariana Grande 80% Full ht...,neutral
1,1,Ariana Grande KIIS FM Yours Truly CD listening...,positive
2,2,Ariana Grande White House Easter Egg Roll in W...,positive
3,3,#CD #Musics Ariana Grande Sweet Like Candy 3.4...,positive
4,4,SIDE TO SIDE 😘 @arianagrande #sidetoside #aria...,neutral
...,...,...,...
12279,12279,@dansen17 update: Zac Efron kissing a puppy ht...,positive
12280,12280,#zac efron sex pic skins michelle sex https://...,neutral
12281,12281,First Look at Neighbors 2 with Zac Efron Shirt...,neutral
12282,12282,zac efron poses nude #lovely libra porn https:...,neutral


In [13]:
sem_eval_test.groupby('Polarity_Classification').size()

Polarity_Classification
negative    3972
neutral     5937
positive    2375
dtype: int64

In [10]:
from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification 
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import torch

In [14]:
def prediction_test(dataset, modello, tok):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    #modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')
    #tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
    test_texts=dataset['Sentence']
    test_labels=dataset['Polarity_Classification']

#unique_labels = set(label for label in test_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}
    unique_labels={'neutral', 'positive', 'negative'}
    label2id={'negative': 2, 'neutral': 0, 'positive': 1}
    id2label={2:'negative', 0:'neutral', 1:'positive'}

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=256)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments( #sono sempre gli stessi alla fine
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels))#,predicted_labels

In [18]:
#augmented 

modello = DistilBertForSequenceClassification.from_pretrained('Polarity_base_new_aug')
tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
prediction_test(sem_eval_test, modello, tok)


loading configuration file Polarity_base_new_aug\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "vocab_size": 119547
}

loading weights file Polarity_base_new_aug\pytorch_model.bin
All model checkpoint weights were used when

              precision    recall  f1-score   support

    negative       0.56      0.24      0.34      3972
     neutral       0.55      0.52      0.54      5937
    positive       0.36      0.75      0.49      2375

    accuracy                           0.48     12284
   macro avg       0.49      0.50      0.45     12284
weighted avg       0.52      0.48      0.46     12284



In [16]:
#m7
#augmented 
#baseline
modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_base_xlm_aug')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
prediction_test(sem_eval_test, modello, tok)

loading configuration file Polarity_base_xlm_aug\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_s

              precision    recall  f1-score   support

    negative       0.60      0.59      0.60      3972
     neutral       0.65      0.46      0.54      5937
    positive       0.45      0.79      0.57      2375

    accuracy                           0.57     12284
   macro avg       0.57      0.61      0.57     12284
weighted avg       0.60      0.57      0.57     12284



In [17]:
#m8

modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_base_xlm_ALSA')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
prediction_test(sem_eval_test, modello, tok)

loading configuration file Polarity_base_xlm_ALSA\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_

              precision    recall  f1-score   support

    negative       0.63      0.50      0.56      3972
     neutral       0.61      0.53      0.57      5937
    positive       0.46      0.76      0.57      2375

    accuracy                           0.56     12284
   macro avg       0.56      0.60      0.56     12284
weighted avg       0.59      0.56      0.56     12284



In [ ]:
#proviamo su un secondo dataset

In [20]:
import pandas as pd
sst=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_test_polarity\\Evaluation_data\\stanfordSentimentTreebank\\sst.csv")
sst


,Unnamed: 0,Sentence,Polarity_Classification
0,0,& - white freeze frames reminiscent of a pseud...,negative
1,1,& Stitch '' is n't the most edgy piece of Disn...,positive
2,2,& Stitch '' reach the emotion or timelessness ...,neutral
3,3,' ( Hopkins ) does n't so much phone in his pe...,negative
4,4,' ( The Cockettes ) provides a window into a s...,positive
...,...,...,...
30014,30014,"yourself staring hypnotically at her , trying ...",neutral
30015,30015,yourself that it 's a '' true story '' and you...,positive
30016,30016,zeal to squeeze the action and our emotions in...,neutral
30017,30017,zips along with B-movie verve while adding the...,positive


In [21]:
#augmented 

modello = DistilBertForSequenceClassification.from_pretrained('Polarity_base_new_aug')
tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
prediction_test(sst, modello, tok)


loading configuration file Polarity_base_new_aug\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "vocab_size": 119547
}

loading weights file Polarity_base_new_aug\pytorch_model.bin
All model checkpoint weights were used when

              precision    recall  f1-score   support

    negative       0.58      0.41      0.48     11183
     neutral       0.34      0.28      0.31      7015
    positive       0.53      0.73      0.62     11821

    accuracy                           0.51     30019
   macro avg       0.49      0.48      0.47     30019
weighted avg       0.51      0.51      0.49     30019



In [22]:
#m7
#augmented 
#baseline
modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_base_xlm_aug')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
prediction_test(sst, modello, tok)

loading configuration file Polarity_base_xlm_aug\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_s

              precision    recall  f1-score   support

    negative       0.71      0.66      0.68     11183
     neutral       0.39      0.29      0.34      7015
    positive       0.67      0.81      0.73     11821

    accuracy                           0.63     30019
   macro avg       0.59      0.59      0.58     30019
weighted avg       0.62      0.63      0.62     30019



In [23]:
#m8

modello = XLMRobertaForSequenceClassification.from_pretrained('Polarity_base_xlm_ALSA')
tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')
prediction_test(sst, modello, tok)

loading configuration file Polarity_base_xlm_ALSA\config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_

              precision    recall  f1-score   support

    negative       0.71      0.67      0.69     11183
     neutral       0.44      0.24      0.31      7015
    positive       0.65      0.85      0.73     11821

    accuracy                           0.64     30019
   macro avg       0.60      0.59      0.58     30019
weighted avg       0.62      0.64      0.62     30019

